# Kapitel 3 - Klimawandel

Den Klimawandel an nur einer einzelnen Wetterstation abzulesen ist nicht ganz leicht. Das liegt vor allem an diesen Gründen:

- die Erde erwärmt sich nicht überall gleich schnell. Und es gibt sogar einige wenige Orte, an denen es über die letzten Jahre eher kälter als wärmer wurde... an diesen Orten kann man also schon prinzipiell Erwärmung finden! Das schauen wir uns später nochmal genauer an.
- im Klimasystem der Erde passiert sehr viel mit unterschiedlichen Zeitskalen: Von Regen, der in 15 Minuten wieder vorbei sein kann, über Hoch- und Tiefdruckgebiete, die sich in Tagen bis Wochen verändern, bis zu sehr langsamen Phänomenen, die Auftreten: vielleicht hast du schonmal vom El Nino gehört - der kommt in etwa alle 2 bis 7 Jahre wiederholt. Man muss aufpassen, dass diese sogenannte "Klima-Variabilität" nicht das Signal beeinflusst, dass man für den Klimawandel bestimmt! Du hast ja mittlerweile auch schon gesehen, dass sehr viel "Rauschen" in den Daten ist.
- last but not least kann es auch mit der Mess-Station Probleme geben: Hat sich das Messgerät verändert? Ist die Station umgezogen? Wurde der Boden / die Umgebung um die Station verändert, so dass sich die Gegend um die Station jetzt zB unter der Sonne mehr aufwärmt?

Also los geht's: Zuerst müssen wir nochmal die Daten laden (kennst du ja schon aus den letzten Kapiteln - deinen Datei +  Ordnernamen eingeben)

In [3]:
# Programmierbibliotheken importieren

import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

In [4]:
d = pd.read_csv(
    "DeinOrdnerName/DeinDateiname",  # "tageswerte_KL_01234_12345678_12345678_hist/produkt_klima_tag_12345678_12345678_01234.txt"
    sep=";",
    skipinitialspace=True,
)

data = xr.DataArray(d["TMK"], coords={"time": d["MESS_DATUM"]})
data["time"] = pd.to_datetime(data.time, format="%Y%m%d")
data = data.sel(time=slice(np.datetime64("1972-01-01"), None))

Damit der Klimawandel nicht von der "Klimavariabilität" überdeckt wird, können wir uns zunächst mal Jahresdurchschnittswerte anschauen:

In [7]:
yearly_averages = data.groupby("time.year").mean()

yearly_averages.plot.scatter()
plt.show()

Was fällt dir auf? Kannst du einen Trend erkennen? Ist das "Erderwärmungs-Signal" klar?

Antwort:

Das der Trend nicht so klar ist überrascht dich vielleicht - das sieht weniger klar aus als die "Klimastreifen", die man oft sieht, um die Erderwärmung zu symbolisieren:

![Klimastreifen](klimastreifen.png) 

(Image Credit: Ed Hawkins)

Werden also alle hinter's Licht geführt von Klimawissenschaftlern? Nein - Denn die Daten in den Klimastreifen sind Jahresdurchschnitte für die gesamte Erde, wärend wir uns nur die Temperatur in Leipzig angeschaut haben. Und wenn man den Durchschnitt über die gesamte Erde nimmt, "mittelt sich viel Rauschen heraus". Du kannst dir zum Beispiel vorstellen, dass es in einem Jahr mit einem sehr heißen Sommer in Deutschland oft einen kalten Sommer anderswo auf der Welt gibt... es ist ein bisschen wie Lotterie spielen, wer ein kaltes und wer ein warmes Jahr bekommt - aber durch den Klimawandel wird es im Durchschnitt für alle wärmer.

## 3.2 Linearer Trend

Du hast vermutlich gesehen, dass es in den Daten der Station Leipzig/Halle tendenziell doch leicht bergauf geht mit der Temperatur. Können wir irgendwie genauer sagen, wie viel das pro Jahr ist?

Um das zu machen schauen wir uns etwas an, was wissenschaftler "Lineare Regression" nennen. Dabei legt man im einfachsten Fall eine Gerade auf die Daten... und verschiebt die dann so lange hin und her, bis der "Fehler", den man durch die gerade macht möglichst klein ist.

Ich hab im nächsten Codeblock mal 3 Geraden definiert - ich denke, man sieht, dass einige davon besser zu den Daten passen als andere:

In [54]:
def f(x, m, b):
    return m * x + b


yearly_averages = data.groupby("time.year").mean()

yearly_averages.plot.scatter(color="k")

plt.plot(yearly_averages.year, f(yearly_averages.year, 0.1, -190))
plt.plot(yearly_averages.year, f(yearly_averages.year, 2, -4000))
plt.plot(yearly_averages.year, f(yearly_averages.year, -0.3, 610))
plt.ylim(0, 20)
plt.show()

Man muss das zum Glück nicht alles von Hand machen - man kann mit Mathe definieren, was es bedeutet, dass eine Gerade den "kleinsten Fehler" macht... und dann kann man ausrechnen, welche Gerade die beste ist. Das machen wir (bzw der Computer) jetzt:

In [ ]:
from scipy import optimize

popt, pcov = optimize.curve_fit(f, yearly_averages.year, yearly_averages)
print(
    "Optimale Steigung: {:.3f}".format(popt[0]),
    "Optimaler y-Achsen-Abschnitt: {:.3f}".format(popt[1]),
)

Schauen wir uns die Gerade an:

In [ ]:
x = yearly_averages.year
y = yearly_averages

plt.scatter(x, y, label="Daten")
plt.plot(
    [min(x), max(x)], [f(min(x), *popt), f(max(x), *popt)], "r--", label="Beste Gerade"
)
plt.legend()

plt.ylim(5, 15)
plt.show()
# yearly_averages.rolling(year=10, center=True).mean().plot()

Das sieht doch gut aus!

Und wenn wir uns die Geradengleichung anschauen können wir auch herausfinden, wie viel es Pro Jahr etwa wärmer wird: Dazu müssen wir auf die Steigung der Geraden schauen:

In [ ]:
"Steigung: {:.3f}".format(popt[0])

Welche Einheiten hat diese Steigung? (Die Antwort ist die Einheit auf der y-Achse, geteilt durch die Einheit auf der x-Achse)

Antwort:

Kannst du herausfinden, 1) wie viel es in 10 Jahren wärmer wird? 2) wie lange es etwa dauern sollte, um 1°C wärmer zu werden?

Antwort 1:
Antwort 2:

Achtung: die Zeit, die wir hier für 1°C zusätzliche Erwärmung ausgerechnet haben muss für die Zukunft nicht richtig sein, nicht einmal für Leipzig! Denn hier haben wir einfach gesagt "die Temperatur folgt über die Zeit einer Geraden" ohne das irgendwie zu begründen. Das ist eine extrem starke Vereinfachung. Und dazu hängt die Zukünftige Klimaveränderung natürlich davon ab, wie viel Treibhausgase ausgestoßen werden - es kann also sein, dass ein zusätzliches °C Erwärmung noch schneller erreicht wird (wenn wir mehr CO2 ausstoßen), oder dass es deutlich länger dauert (wenn wir den Klimawandel effektiv bekämpfen) - oder überhaupt nie erreicht wird.

## 3.3 Vergleich mit weltweitem Klimawandel

Bis jetzt haben wir uns nur die Daten einer einzelnen Wetterstation angeschaut - wie Unterschiedlich ist unser Ergebnis von der Klimaveränderung in ganz Deutschland? Und wie unterschiedlich Verhält es sich an verschiedenen Orten der Erde?

Der Weltklimarat hat einen [Atlas](https://interactive-atlas.ipcc.ch/regional-information#eyJ0eXBlIjoiQVRMQVMiLCJjb21tb25zIjp7ImxhdCI6OTc3MiwibG5nIjo0MDA2OTIsInpvb20iOjQsInByb2oiOiJFUFNHOjU0MDMwIiwibW9kZSI6ImNvbXBsZXRlX2F0bGFzIn0sInByaW1hcnkiOnsic2NlbmFyaW8iOiJvYnNlcnZhdGlvbnMiLCJwZXJpb2QiOiIxOTgwLTIwMTUiLCJzZWFzb24iOiJ5ZWFyIiwiZGF0YXNldCI6Ik9CUy1FUkE1IiwidmFyaWFibGUiOiJ0YXMiLCJ2YWx1ZVR5cGUiOiJUUkVORCIsImhhdGNoaW5nIjoiRElTQUJMRUQiLCJyZWdpb25TZXQiOiJhcjYiLCJiYXNlbGluZSI6InByZUluZHVzdHJpYWwiLCJyZWdpb25zU2VsZWN0ZWQiOltdfSwicGxvdCI6eyJhY3RpdmVUYWIiOiJwbHVtZSIsIm1hc2siOiJub25lIiwic2NhdHRlcllNYWciOm51bGwsInNjYXR0ZXJZVmFyIjpudWxsLCJzaG93aW5nIjpmYWxzZX19) veröffentlicht, auf dem man sich weltweite Temperatur-Beobachtungsdaten (und Projektionen für die Zukunft unter verschiedenen Szenarien) anschauen kann.

Schau dir die Karte an: Wie viel hat sich die Temperatur in der Gegend von Leipzig (muss nicht genau stimmen) zwischen 1980 und 2015 in etwa pro Dekade (10 Jahre) erwärmt? Vergleiche mit unserem Ergebnis!

Antwort:

Was fällt dir sonst so auf? Welche Regionen haben sich besonders start erwärmt, welche nur sehr wenig oder sind sogar kälter geworden? Was sind die höchsten und niedrigsten Werte, die du finden kannst? (du kannst auch die Farbskala in dem Atlas einstellen indem du auf das Symbol mit dem Stift über der Farbskala klickst)

Antwort:

Größte Erwärmung pro Dekade: °C/10 Jahre
Geringste Erwärmung/stärkste Abkühlung pro Dekade: °C/10 Jahre

Damit sind wir auch mit diesem Kapitel fertig! Weiter geht's mit [Kapitel 4](Kapitel-4-Spektrum.ipynb).